### Generate behaviour video

You need to link two timestamp files, one will pertain to the framerate of the behaviour cam and the other of the calcium imaging video. We won't be synchronizing them framerate wise as long as they roughly finish and end at the same time we'll be fine. This has been setup to just work with avi files but you can look at the minian repo if you need a more comprehensive implementation.

In [ ]:
from caltrig.core.backend import open_minian, overwrite_xarray
import pandas as pd

ImportError: cannot import name 'save_xarray' from 'caltrig.core.backend' (c:\Users\Michal Lange\Documents\Calcium-Transient-Analysis\caltrig\core\backend.py)

In [24]:
minian_timestamps_path = "/N/project/Cortical_Calcium_Image/Miniscope data/05.2023_Tenth_group/AA058_D1/AA058_D1_S1_behavior_ms.csv"
behavior_timestamps_path = "/N/project/Cortical_Calcium_Image/Miniscope data/05.2023_Tenth_group/AA058_D1/2023_05_05/11_02_42/BehavCam_0/timeStamps.csv"
behavior_video_path = "/N/project/Cortical_Calcium_Image/Miniscope data/05.2023_Tenth_group/AA058_D1/2023_05_05/11_02_42/BehavCam_0/minian_intermediate_behavior"

In [25]:
result = open_minian(behavior_video_path).to_dataarray(name="behavior_video_S1")

/geode2/home/u060/milange/Quartz/Documents/Cell_Clustering_Tool/core/backend.py:87: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  arr = list(xr.open_zarr(arr_path).values())[0]


In [26]:
result

<xarray.DataArray 'behavior_video_S1' (variable: 1, frame: 53946, height: 480,
                                       width: 640)>
dask.array<stack, shape=(1, 53946, 480, 640), dtype=uint8, chunksize=(1, 100, 480, 640), chunktype=numpy.ndarray>
Coordinates:
  * frame     (frame) int64 0 1 2 3 4 5 ... 53940 53941 53942 53943 53944 53945
  * height    (height) int64 0 1 2 3 4 5 6 7 ... 472 473 474 475 476 477 478 479
  * width     (width) int64 0 1 2 3 4 5 6 7 ... 632 633 634 635 636 637 638 639
  * variable  (variable) object 'behavior_cam'

In [35]:
# Read both timestamps into a pandas dataframe
minian_timestamps = pd.read_csv(minian_timestamps_path)
# Get maximum of Time Stamp (ms) column. WARNING for some videos you'll also need to account for a different start time as well.
minian_max = minian_timestamps["Time Stamp (ms)"].max()

behavior_timestamps = pd.read_csv(behavior_timestamps_path)
# Subtract minian_max from Time Stamp (ms) column take the absolute value and get the index of the minimum value
behavior_timestamps["Time Stamp (ms)"] = abs(behavior_timestamps["Time Stamp (ms)"] - minian_max)
final_frame = behavior_timestamps["Time Stamp (ms)"].idxmin()

In [41]:
# Open the video and slice it to the current frame
new_varr = result.sel(frame=(0, final_frame))
da = save_xarray(result, "/N/project/Cortical_Calcium_Image/Miniscope data/05.2023_Tenth_group/AA058_D1/2023_05_05/11_02_42/BehavCam_0/minian_intermediate_behavior")

KeyError: "not all values found in index 'frame'. Try setting the `method` keyword argument (example: method='nearest')."

In [42]:
for i in range(0, final_frame):
    if i not in result.coords.get("frame").values:
        print(i)


array([    0,     1,     2, ..., 53943, 53944, 53945])

In [40]:
da

<xarray.DataArray 'behavior_video_S1' (variable: 1, frame: 53946, height: 480,
                                       width: 640)>
dask.array<from-zarr, shape=(1, 53946, 480, 640), dtype=uint8, chunksize=(1, 100, 480, 640), chunktype=numpy.ndarray>
Coordinates:
  * frame     (frame) int64 0 1 2 3 4 5 ... 53940 53941 53942 53943 53944 53945
  * height    (height) int64 0 1 2 3 4 5 6 7 ... 472 473 474 475 476 477 478 479
  * variable  (variable) object 'behavior_cam'
  * width     (width) int64 0 1 2 3 4 5 6 7 ... 632 633 634 635 636 637 638 639